# Segmenting and Clustering Neighborhoods in Toronto

## Part 1

### Dataframe of the Toronto Neighborhoods and Boroughs

In [1]:
!pip install bs4
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
# url to extract the data
url = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1011037969"

In [3]:
# Create the DataFrame of the neighborhoods of Toronto
Toronto_Neighborhoods=pd.read_html(url,match="Postal Code",flavor='bs4')[0]
Toronto_Neighborhoods.columns = ['PostalCode', 'Borough', 'Neighborhood']
Toronto_Neighborhoods.shape

(180, 3)

In [4]:
# Ignore cells with a borough that is Not assigned
Toronto_Neighborhoods = Toronto_Neighborhoods[Toronto_Neighborhoods['Borough'] != 'Not assigned']
Toronto_Neighborhoods.shape

(103, 3)

In [5]:
# If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.
Neighborhood_Not_Assigned = Toronto_Neighborhoods["Neighborhood"] == "Not assigned"
Toronto_Neighborhoods.loc[Neighborhood_Not_Assigned, "Neighborhood"] = \
Toronto_Neighborhoods.loc[Neighborhood_Not_Assigned, "Borough"]
Toronto_Neighborhoods.reset_index(drop=True,inplace=True)
Toronto_Neighborhoods.shape

(103, 3)

## Part 2

### Merge with coordinates

In [6]:
# @hidden_cell
filename = '/home/nicolas/Coursera/Data Science/Capstone Project/Geospatial_data.csv'

In [7]:
# Download and save CSV file
url = 'http://cocl.us/Geospatial_data'
myfile = requests.get(url, allow_redirects=True)
open(filename, 'wb').write(myfile.content)

2891

In [8]:
df = pd.read_csv(filename)
df.columns = ['PostalCode', 'Latitude', 'Longitude']

In [9]:
merged_Frame = pd.merge(Toronto_Neighborhoods,df, on = 'PostalCode',how='inner')
merged_Frame.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
